In [3]:
import os
import json
import warnings 
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO

# 시각화를 위한 라이브러리
from ipywidgets import interact
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.patches import Patch

#!pip install webcolors
import webcolors

In [4]:
class_colormap = pd.read_csv("/opt/ml/input/code/class_dict.csv")
result_1 = pd.read_csv("/opt/ml/input/code/submission/cv_val.csv")
result_2 = pd.read_csv("/opt/ml/input/code/submission/swin_L(pretrained).csv")
anns_file_path = '/opt/ml/check_all/inspect_1 (1).json'
coco = COCO(anns_file_path)
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories_gt = dataset['categories']
anns_gt = dataset['annotations']
imgs_gt = dataset['images']
imgs_pr_1 = result_1["image_id"].tolist()
imgs_pr_2 = result_2["image_id"].tolist()

colormap = np.zeros((11, 3), dtype=np.uint8)
for inex, (_, r, g, b) in enumerate(class_colormap.values):
    colormap[inex] = [r, g, b]

loading annotations into memory...
Done (t=1.52s)
creating index...
index created!


## 시각화

In [7]:
@interact(idx=(675, len(imgs_gt)-1))
def plot_csv(idx):
    image_gt = imgs_gt[idx]["file_name"]
    mask_gt = np.zeros((512, 512))
    for ann in anns_gt:
        if ann['image_id'] == imgs_gt[idx]['id']:
            pixel_value = ann["category_id"]
            mask_gt[coco.annToMask(ann) == 1] = pixel_value
            mask_gt = mask_gt.astype(np.int8)
    
    f = lambda x: colormap[x]
    mask_gt = f(mask_gt)
    
    if image_gt in imgs_pr_1:
        result = result_1
        j = imgs_pr_1.index(image_gt)
    else:
        result = result_2
        j = imgs_pr_2.index(image_gt)
    mask = result['PredictionString'][j]
    mask = list(map(lambda x: colormap[int(x)], mask.split()))
    mask = [mask[i:i+512] for i in range(0, len(mask), 512)]
    
    fig, ax = plt.subplots(ncols=3, figsize=(20, 20))
    category_and_rgb = [[category, (r,g,b)] for idx, (category, r, g, b) in enumerate(class_colormap.values)]
    legend_elements = [Patch(facecolor=webcolors.rgb_to_hex(rgb), 
                             edgecolor=webcolors.rgb_to_hex(rgb), 
                             label=category) for category, rgb in category_and_rgb]
    # Original Image
    img = Image.open(os.path.join('/opt/ml/input/data',image_gt))
    ax[0].imshow(img)
    ax[0].set_title(f"Orignal Image : {image_gt}")
    ax[0].axis('off')
    # Pred Mask
    ax[1].imshow(mask_gt)
    ax[1].set_title(f"Ground Truth")
    ax[1].axis('off')
    # Pred Mask
    ax[2].imshow(mask)
    ax[2].set_title(f"Pred Mask")
    ax[2].legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
    ax[2].axis('off')
    
    
    plt.grid(False)
    plt.show()

interactive(children=(IntSlider(value=811, description='idx', max=948, min=675), Output()), _dom_classes=('wid…